<a href="https://colab.research.google.com/github/sittidetw/dads5001_midterm_project/blob/main/Data_Prep_HIV_Proactive_Screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sittidetw/dads5001_midterm_project.git

Cloning into 'dads5001_midterm_project'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 179 (delta 61), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (179/179), 1.36 MiB | 6.71 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [2]:
pip install python-calamine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.1/935.1 kB 8.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import gc

In [4]:
df_province = pd.read_excel('/content/dads5001_midterm_project/prep_data/master_province.xlsx', engine='calamine')
for col in df_province.iloc[0:,0:].columns: df_province[col] = df_province[col].astype('string')
display(df_province.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   health_region          77 non-null     string
 1   province_th            77 non-null     string
 2   health_region_desc_th  77 non-null     string
 3   region_th              77 non-null     string
 4   province               77 non-null     string
 5   health_region_desc     77 non-null     string
 6   region                 77 non-null     string
dtypes: string(7)
memory usage: 4.3 KB


None

# Data Cleaning

In [5]:
# @title HIV Testing by Key Populations
path = "/content/dads5001_midterm_project/rd_testing_population/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_test_pop_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[2]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.contains('testing')) & (df[1].str.contains('Female')),
      (df[0].str.contains('testing')) & (df[1].str.contains('Male')),
      (df[0].str.contains('testing')) & (df[1].str.contains('All KP')),
      (df[0].str.contains('testing')) & (df[1].str.contains('Total')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Female')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Male')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('All KP')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Total'))
  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'test_female', 'test_male', 'test_all_kp', 'test_total',
            'positive_female', 'positive_male', 'positive_all_kp', 'positive_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_test_pop_combine is None):
    df_test_pop_combine = df
  else:
    df_test_pop_combine = pd.concat([df_test_pop_combine, df], ignore_index=True)

display(df_test_pop_combine.groupby(['year','quarter'])['positive_grand_total'].count())

# Transform accumulated data to total year summary
df_test_pop = df_test_pop_combine[df_test_pop_combine['q'] == 'Q4'].copy()
df_test_pop_temp1 = df_test_pop_combine[(df_test_pop_combine['q'] == 'Q1') & (df_test_pop_combine['year'].isin(df_test_pop['year'].unique()))].copy()

next_years_int = df_test_pop['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_test_pop_temp2 = df_test_pop_combine[(df_test_pop_combine['q'] == 'Q1') & (df_test_pop_combine['year'].isin(next_years_str))].copy()

df_test_pop = df_test_pop.set_index(['year', 'province'])
df_test_pop_temp1 = df_test_pop_temp1.set_index(['year', 'province'])

df_test_pop_temp2['year'] = df_test_pop_temp2['year'].astype(int) - 1
df_test_pop_temp2['year'] = df_test_pop_temp2['year'].astype(str)
df_test_pop_temp2 = df_test_pop_temp2.set_index(['year', 'province'])

cols_to_cal = df_test_pop.columns[3:]
#df_test_pop[cols_to_cal] = (df_test_pop[cols_to_cal] - df_test_pop_temp1[cols_to_cal]) + df_test_pop_temp2[cols_to_cal]
df_test_pop[cols_to_cal] = df_test_pop[cols_to_cal].sub(df_test_pop_temp1[cols_to_cal], fill_value=0).add(df_test_pop_temp2[cols_to_cal], fill_value=0)

df_test_pop = df_test_pop.reset_index(drop=False)
df_test_pop = df_test_pop.drop(columns=['q','quarter'])
df_test_pop.iloc[:, 3:] = df_test_pop.iloc[:, 3:].clip(lower=0)

# Delete temp variables
del df_test_pop_temp1
del df_test_pop_temp2
gc.collect()

test_male = [col for col in df_test_pop.columns if col.startswith('test_male_') and 'all_kps' not in col and 'total' not in col]
test_female = [col for col in df_test_pop.columns if col.startswith('test_female_') and 'all_kps' not in col and 'total' not in col]
positive_male = [col for col in df_test_pop.columns if col.startswith('positive_male_') and 'all_kps' not in col and 'total' not in col]
positive_female = [col for col in df_test_pop.columns if col.startswith('positive_female_') and 'all_kps' not in col and 'total' not in col]

df_test_pop['test_male_all_kps'] = df_test_pop[test_male].sum(axis=1)
df_test_pop['test_female_all_kps'] = df_test_pop[test_female].sum(axis=1)
df_test_pop['positive_male_all_kps'] = df_test_pop[positive_male].sum(axis=1)
df_test_pop['positive_female_all_kps'] = df_test_pop[positive_female].sum(axis=1)

# Add sum of others
added_col = ['test_male_others','test_female_others','positive_male_others','positive_female_others']
col_name = ['test_male_all_kps','test_female_all_kps','positive_male_all_kps','positive_female_all_kps']
col_ttl = ['test_male_total','test_female_total','positive_male_total','positive_female_total']
indices = df_test_pop.columns.get_indexer(col_name)
for i in reversed(range(len(indices))):
    indice = indices[i]
    new_col = added_col[i]
    source_ttl = col_ttl[i]
    source_kp = col_name[i]
    df_test_pop.insert(loc=indice + 1, column=new_col, value=df_test_pop[source_ttl] - df_test_pop[source_kp])

df_test_pop.iloc[:, 3:] = df_test_pop.iloc[:, 3:].clip(lower=0)
df_test_pop['test_male_total'] = df_test_pop[['test_male_all_kps','test_male_others']].sum(axis=1)
df_test_pop['test_female_total'] = df_test_pop[['test_female_all_kps','test_female_others']].sum(axis=1)
df_test_pop['positive_male_total'] = df_test_pop[['positive_male_all_kps','positive_male_others']].sum(axis=1)
df_test_pop['positive_female_total'] = df_test_pop[['positive_female_all_kps','positive_female_others']].sum(axis=1)

df_test_pop = df_test_pop.drop(columns=['test_all_kp_total','positive_all_kp_total','test_grand_total','positive_grand_total'])

# Melt to Long Format (Unpivot Columns)
df_test_pop = df_test_pop.melt(
    id_vars=['year', 'province', 'health_region'],
    var_name='temp_column',
    value_name='count'
)

split_data = df_test_pop['temp_column'].str.split('_', expand=True)
df_test_pop['sex'] = split_data[1]

df_test_pop['category'] = split_data[0] + '_' + split_data.iloc[:, 2:].fillna('').agg('_'.join, axis=1).str.strip('_')
df_test_pop = df_test_pop[['year', 'province', 'health_region', 'sex', 'category', 'count']]

# Pivot to Wide Format
df_test_pop = df_test_pop.pivot(
    index=['year', 'province', 'health_region', 'sex'],
    columns='category',
    values='count'
).reset_index()
df_test_pop.columns.name = None

# Change Type
for col in df_test_pop.iloc[0:,0:4].columns: df_test_pop[col] = df_test_pop[col].astype('string')
#for col in df_test_pop.iloc[0:,4:].columns: df_test_pop[col] = df_test_pop[col].astype('int64')

# Add spatial information
df_test_pop = df_test_pop.rename(columns={'province':'province_th'})
df_test_pop = df_test_pop.merge(df_province, on=['health_region', 'province_th'], how='left')

# Sort Columns
desired_order = ['year', 'health_region', 'province_th', 'health_region_desc_th', 'region_th', 'province', 'health_region_desc', 'region', 'sex',
                 'test_fsw', 'test_migrant', 'test_msm', 'test_msw', 'test_partner_of_hiv', 'test_prisoner', 'test_pwid', 'test_tg', 'test_tgsw', 'test_all_kps', 'test_others', 'test_total',
                 'positive_fsw', 'positive_migrant', 'positive_msm', 'positive_msw', 'positive_partner_of_hiv', 'positive_prisoner', 'positive_pwid', 'positive_tg', 'positive_tgsw', 'positive_all_kps', 'positive_others', 'positive_total'
]
actual_columns = [col for col in desired_order if col in df_test_pop.columns]
df_test_pop = df_test_pop[actual_columns]

# Sort Rows
def multi_column_sorter(col):
    if col.name == 'health_region':
        return col.astype(int)
    return col

df_test_pop = df_test_pop.sort_values(by=['year','health_region','province_th'], key=multi_column_sorter).reset_index(drop=True)

display(df_test_pop.head())
display(df_test_pop.info())
df_test_pop.to_excel('test_pop.xlsx', index=False)

Total files: 11
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2023Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2026Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2021Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2021Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2024Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2025Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2022Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2022Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2023Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2025Q4_20260215.xlsx
/content/dads5001_midter

year  quarter
2021  2021Q1     77
      2021Q4     77
2022  2022Q1     77
      2022Q4     77
2023  2023Q1     77
      2023Q4     77
2024  2024Q1     77
      2024Q4     77
2025  2025Q1     77
      2025Q4     77
2026  2026Q1     77
Name: positive_grand_total, dtype: int64

,year,health_region,province_th,health_region_desc_th,region_th,province,health_region_desc,region,sex,test_fsw,...,positive_msm,positive_msw,positive_partner_of_hiv,positive_prisoner,positive_pwid,positive_tg,positive_tgsw,positive_all_kps,positive_others,positive_total
0,2021,1,น่าน,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Nan,Upper Northern (Lanna),North,female,0,...,0,0,2,0,0,0,0,2,20,22
1,2021,1,น่าน,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Nan,Upper Northern (Lanna),North,male,0,...,24,0,0,2,0,1,0,27,27,54
2,2021,1,พะเยา,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Phayao,Upper Northern (Lanna),North,female,171,...,0,0,6,0,0,0,0,7,24,31
3,2021,1,พะเยา,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Phayao,Upper Northern (Lanna),North,male,0,...,36,1,4,1,0,1,0,43,52,95
4,2021,1,ลำปาง,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Lampang,Upper Northern (Lanna),North,female,2,...,0,0,3,0,0,0,0,3,39,42


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   year                     770 non-null    string
 1   health_region            770 non-null    string
 2   province_th              770 non-null    string
 3   health_region_desc_th    770 non-null    string
 4   region_th                770 non-null    string
 5   province                 770 non-null    string
 6   health_region_desc       770 non-null    string
 7   region                   770 non-null    string
 8   sex                      770 non-null    string
 9   test_fsw                 770 non-null    int64 
 10  test_migrant             770 non-null    int64 
 11  test_msm                 770 non-null    int64 
 12  test_msw                 770 non-null    int64 
 13  test_partner_of_hiv      770 non-null    int64 
 14  test_prisoner            770 non-null    i

None

In [6]:
# @title Reach Populations
path = "/content/dads5001_midterm_project/rd_reach_population/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_reach_pop_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[2]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.contains('Reach')) & (df[1].str.contains('Female')),
      (df[0].str.contains('Reach')) & (df[1].str.contains('Male')),
      (df[0].str.contains('Reach')) & (df[1].str.contains('All KP')),
      (df[0].str.contains('Reach')) & (df[1].str.contains('Total')),

  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'reach_female', 'reach_male', 'reach_all_kp', 'reach_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_reach_pop_combine is None):
    df_reach_pop_combine = df
  else:
    df_reach_pop_combine = pd.concat([df_reach_pop_combine, df], ignore_index=True)

display(df_reach_pop_combine.groupby(['year','quarter'])['reach_grand_total'].count())

# Transform accumulated data to total year summary
df_reach_pop = df_reach_pop_combine[df_reach_pop_combine['q'] == 'Q4'].copy()
df_reach_pop_temp1 = df_reach_pop_combine[(df_reach_pop_combine['q'] == 'Q1') & (df_reach_pop_combine['year'].isin(df_reach_pop['year'].unique()))].copy()

next_years_int = df_reach_pop['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_reach_pop_temp2 = df_reach_pop_combine[(df_reach_pop_combine['q'] == 'Q1') & (df_reach_pop_combine['year'].isin(next_years_str))].copy()

df_reach_pop = df_reach_pop.set_index(['year', 'province'])
df_reach_pop_temp1 = df_reach_pop_temp1.set_index(['year', 'province'])

df_reach_pop_temp2['year'] = df_reach_pop_temp2['year'].astype(int) - 1
df_reach_pop_temp2['year'] = df_reach_pop_temp2['year'].astype(str)
df_reach_pop_temp2 = df_reach_pop_temp2.set_index(['year', 'province'])

cols_to_cal = df_reach_pop.columns[3:]
#df_reach_pop[cols_to_cal] = (df_reach_pop[cols_to_cal] - df_reach_pop_temp1[cols_to_cal]) + df_reach_pop_temp2[cols_to_cal]
df_reach_pop[cols_to_cal] = df_reach_pop[cols_to_cal].sub(df_reach_pop_temp1[cols_to_cal], fill_value=0).add(df_reach_pop_temp2[cols_to_cal], fill_value=0)

df_reach_pop = df_reach_pop.reset_index(drop=False)
df_reach_pop = df_reach_pop.drop(columns=['q','quarter'])
df_reach_pop.iloc[:, 3:] = df_reach_pop.iloc[:, 3:].clip(lower=0)

# Delete temp variables
del df_reach_pop_temp1
del df_reach_pop_temp2
gc.collect()

test_male = [col for col in df_reach_pop.columns if col.startswith('reach_male_') and 'all_kps' not in col and 'total' not in col]
test_female = [col for col in df_reach_pop.columns if col.startswith('reach_female_') and 'all_kps' not in col and 'total' not in col]

df_reach_pop['reach_male_all_kps'] = df_reach_pop[test_male].sum(axis=1)
df_reach_pop['reach_female_all_kps'] = df_reach_pop[test_female].sum(axis=1)

# Add sum of others
added_col = ['reach_male_others','reach_female_others']
col_name = ['reach_male_all_kps','reach_female_all_kps']
col_ttl = ['reach_male_total','reach_female_total']
indices = df_reach_pop.columns.get_indexer(col_name)
for i in reversed(range(len(indices))):
    indice = indices[i]
    new_col = added_col[i]
    source_ttl = col_ttl[i]
    source_kp = col_name[i]
    df_reach_pop.insert(loc=indice + 1, column=new_col, value=df_reach_pop[source_ttl] - df_reach_pop[source_kp])

df_reach_pop.iloc[:, 3:] = df_reach_pop.iloc[:, 3:].clip(lower=0)
df_reach_pop['reach_male_total'] = df_reach_pop[['reach_male_all_kps','reach_male_others']].sum(axis=1)
df_reach_pop['reach_female_total'] = df_reach_pop[['reach_female_all_kps','reach_female_others']].sum(axis=1)

df_reach_pop = df_reach_pop.drop(columns=['reach_all_kp_total','reach_grand_total'])

# Melt to Long Format (Unpivot Columns)
df_reach_pop = df_reach_pop.melt(
    id_vars=['year', 'province', 'health_region'],
    var_name='temp_column',
    value_name='count'
)

split_data = df_reach_pop['temp_column'].str.split('_', expand=True)
df_reach_pop['sex'] = split_data[1]

df_reach_pop['category'] = split_data[0] + '_' + split_data.iloc[:, 2:].fillna('').agg('_'.join, axis=1).str.strip('_')
df_reach_pop = df_reach_pop[['year', 'province', 'health_region', 'sex', 'category', 'count']]

# Pivot to Wide Format
df_reach_pop = df_reach_pop.pivot(
    index=['year', 'province', 'health_region', 'sex'],
    columns='category',
    values='count'
).reset_index()
df_reach_pop.columns.name = None

# Change Type
for col in df_reach_pop.iloc[0:,0:4].columns: df_reach_pop[col] = df_reach_pop[col].astype('string')
for col in df_reach_pop.iloc[0:,4:].columns: df_reach_pop[col] = df_reach_pop[col].astype('int64')

# Add spatial information
df_reach_pop = df_reach_pop.rename(columns={'province':'province_th'})
df_reach_pop = df_reach_pop.merge(df_province, on=['health_region', 'province_th'], how='left')

# Sort Columns
desired_order = ['year', 'health_region', 'province_th', 'health_region_desc_th', 'region_th', 'province', 'health_region_desc', 'region', 'sex',
                 'reach_fsw', 'reach_migrant', 'reach_msm', 'reach_msw', 'reach_partner_of_hiv', 'reach_prisoner', 'reach_pwid', 'reach_tg', 'reach_tgsw', 'reach_all_kps', 'reach_others', 'reach_total'
]
actual_columns = [col for col in desired_order if col in df_reach_pop.columns]
df_reach_pop = df_reach_pop[actual_columns]

# Sort Rows
def multi_column_sorter(col):
    if col.name == 'health_region':
        return col.astype(int)
    return col

df_reach_pop = df_reach_pop.sort_values(by=['year','health_region','province_th'], key=multi_column_sorter).reset_index(drop=True)

display(df_reach_pop.head())
display(df_reach_pop.info())
df_reach_pop.to_excel('reach_pop.xlsx', index=False)

Total files: 7
/content/dads5001_midterm_project/rd_reach_population/REACH_Population_2025Q1_20260221.xlsx
/content/dads5001_midterm_project/rd_reach_population/REACH_Population_2024Q4_20260221.xlsx
/content/dads5001_midterm_project/rd_reach_population/REACH_Population_2026Q1_20260221.xlsx
/content/dads5001_midterm_project/rd_reach_population/REACH_Population_2024Q1_20260221.xlsx
/content/dads5001_midterm_project/rd_reach_population/REACH_Population_2025Q4_20260221.xlsx
/content/dads5001_midterm_project/rd_reach_population/REACH_Population_2023Q4_20260221.xlsx
/content/dads5001_midterm_project/rd_reach_population/REACH_Population_2023Q1_20260221.xlsx


year  quarter
2023  2023Q1     59
      2023Q4     59
2024  2024Q1     57
      2024Q4     57
2025  2025Q1     49
      2025Q4     60
2026  2026Q1     44
Name: reach_grand_total, dtype: int64

,year,health_region,province_th,health_region_desc_th,region_th,province,health_region_desc,region,sex,reach_fsw,reach_migrant,reach_msm,reach_msw,reach_prisoner,reach_pwid,reach_tg,reach_tgsw,reach_all_kps,reach_others,reach_total
0,2023,1,น่าน,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Nan,Upper Northern (Lanna),North,female,522,0,0,0,0,0,0,0,522,0,522
1,2023,1,น่าน,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Nan,Upper Northern (Lanna),North,male,0,0,1299,0,0,0,144,0,1443,0,1443
2,2023,1,พะเยา,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Phayao,Upper Northern (Lanna),North,female,321,0,1,0,0,0,0,0,322,0,322
3,2023,1,พะเยา,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Phayao,Upper Northern (Lanna),North,male,0,0,3711,0,0,0,0,0,3711,0,3711
4,2023,1,ลำพูน,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Lamphun,Upper Northern (Lanna),North,female,0,0,0,0,0,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   year                   352 non-null    string
 1   health_region          352 non-null    string
 2   province_th            352 non-null    string
 3   health_region_desc_th  352 non-null    string
 4   region_th              352 non-null    string
 5   province               352 non-null    string
 6   health_region_desc     352 non-null    string
 7   region                 352 non-null    string
 8   sex                    352 non-null    string
 9   reach_fsw              352 non-null    int64 
 10  reach_migrant          352 non-null    int64 
 11  reach_msm              352 non-null    int64 
 12  reach_msw              352 non-null    int64 
 13  reach_prisoner         352 non-null    int64 
 14  reach_pwid             352 non-null    int64 
 15  reach_tg               

None

In [7]:
# @title PrEP Cur Populations
path = "/content/dads5001_midterm_project/rd_prep_population/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_prep_pop_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[3]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Female')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Male')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('All KP')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Total')),
  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'prep_female', 'prep_male', 'prep_all_kp', 'prep_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_prep_pop_combine is None):
    df_prep_pop_combine = df
  else:
    df_prep_pop_combine = pd.concat([df_prep_pop_combine, df], ignore_index=True)

display(df_prep_pop_combine.groupby(['year','quarter'])['prep_grand_total'].count())

# Transform accumulated data to total year summary
df_prep_pop = df_prep_pop_combine[df_prep_pop_combine['q'] == 'Q4'].copy()
df_prep_pop_temp1 = df_prep_pop_combine[(df_prep_pop_combine['q'] == 'Q1') & (df_prep_pop_combine['year'].isin(df_prep_pop['year'].unique()))].copy()

next_years_int = df_prep_pop['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_prep_pop_temp2 = df_prep_pop_combine[(df_prep_pop_combine['q'] == 'Q1') & (df_prep_pop_combine['year'].isin(next_years_str))].copy()

df_prep_pop = df_prep_pop.set_index(['year', 'province'])
df_prep_pop_temp1 = df_prep_pop_temp1.set_index(['year', 'province'])

df_prep_pop_temp2['year'] = df_prep_pop_temp2['year'].astype(int) - 1
df_prep_pop_temp2['year'] = df_prep_pop_temp2['year'].astype(str)
df_prep_pop_temp2 = df_prep_pop_temp2.set_index(['year', 'province'])

cols_to_cal = df_prep_pop.columns[3:]
#df_prep_pop[cols_to_cal] = (df_prep_pop[cols_to_cal] - df_prep_pop_temp1[cols_to_cal]) + df_prep_pop_temp2[cols_to_cal]
df_prep_pop[cols_to_cal] = df_prep_pop[cols_to_cal].sub(df_prep_pop_temp1[cols_to_cal], fill_value=0).add(df_prep_pop_temp2[cols_to_cal], fill_value=0)

df_prep_pop = df_prep_pop.reset_index(drop=False)
df_prep_pop = df_prep_pop.drop(columns=['q','quarter'])
df_prep_pop.iloc[:, 3:] = df_prep_pop.iloc[:, 3:].clip(lower=0)
df_prep_pop = df_prep_pop.drop(columns=['prep_all_kp_total','prep_grand_total'])

# Delete temp variables
del df_prep_pop_temp1
del df_prep_pop_temp2
gc.collect()

prep_male = [col for col in df_prep_pop.columns if col.startswith('prep_male_') and 'all_kps' not in col and 'total' not in col]
prep_female = [col for col in df_prep_pop.columns if col.startswith('prep_female_') and 'all_kps' not in col and 'total' not in col]

df_prep_pop['prep_male_all_kps'] = df_prep_pop[prep_male].sum(axis=1)
df_prep_pop['prep_female_all_kps'] = df_prep_pop[prep_female].sum(axis=1)

# Add sum of others
added_col = ['prep_male_others','prep_female_others']
col_name = ['prep_male_all_kps','prep_female_all_kps']
col_ttl = ['prep_male_total','prep_female_total']
indices = df_prep_pop.columns.get_indexer(col_name)
for i in reversed(range(len(indices))):
    indice = indices[i]
    new_col = added_col[i]
    source_ttl = col_ttl[i]
    source_kp = col_name[i]
    df_prep_pop.insert(loc=indice + 1, column=new_col, value=df_prep_pop[source_ttl] - df_prep_pop[source_kp])
    df_prep_pop[new_col] = df_prep_pop[new_col].clip(lower=0)

df_prep_pop.iloc[:, 3:] = df_prep_pop.iloc[:, 3:].clip(lower=0)
df_prep_pop['prep_male_total'] = df_prep_pop[['prep_male_all_kps','prep_male_others']].sum(axis=1)
df_prep_pop['prep_female_total'] = df_prep_pop[['prep_female_all_kps','prep_female_others']].sum(axis=1)

# Melt to Long Format (Unpivot Columns)
df_prep_pop = df_prep_pop.melt(
    id_vars=['year', 'province', 'health_region'],
    var_name='temp_column',
    value_name='count'
)

split_data = df_prep_pop['temp_column'].str.split('_', expand=True)
df_prep_pop['sex'] = split_data[1]

df_prep_pop['category'] = split_data[0] + '_' + split_data.iloc[:, 2:].fillna('').agg('_'.join, axis=1).str.strip('_')
df_prep_pop = df_prep_pop[['year', 'province', 'health_region', 'sex', 'category', 'count']]

# Pivot to Wide Format
df_prep_pop = df_prep_pop.pivot(
    index=['year', 'province', 'health_region', 'sex'],
    columns='category',
    values='count'
).reset_index()
df_prep_pop.columns.name = None

# Change Type
for col in df_prep_pop.iloc[0:,0:4].columns: df_prep_pop[col] = df_prep_pop[col].astype('string')
for col in df_prep_pop.iloc[0:,4:].columns: df_prep_pop[col] = df_prep_pop[col].astype('int64')

# Add spatial information
df_prep_pop = df_prep_pop.rename(columns={'province':'province_th'})
df_prep_pop = df_prep_pop.merge(df_province, on=['health_region', 'province_th'], how='left')

# Sort Columns
desired_order = ['year', 'health_region', 'province_th', 'health_region_desc_th', 'region_th', 'province', 'health_region_desc', 'region', 'sex',
                 'prep_fsw', 'prep_migrant', 'prep_msm', 'prep_msw', 'prep_prisoner', 'prep_pwid', 'prep_tg', 'prep_tgsw', 'prep_all_kps', 'prep_others', 'prep_total']
actual_columns = [col for col in desired_order if col in df_prep_pop.columns]
df_prep_pop = df_prep_pop[actual_columns]

# Sort Rows
def multi_column_sorter(col):
    if col.name == 'health_region':
        return col.astype(int)
    return col

display(df_prep_pop.head())
display(df_prep_pop.info())
df_prep_pop.to_excel('prep_pop.xlsx', index=False)

Total files: 7
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2023Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2024Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2026Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2025Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2025Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2024Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2023Q4_20260215.xlsx


year  quarter
2023  2023Q1     54
      2023Q4     67
2024  2024Q1     74
      2024Q4     74
2025  2025Q1     72
      2025Q4     76
2026  2026Q1     69
Name: prep_grand_total, dtype: int64

,year,health_region,province_th,health_region_desc_th,region_th,province,health_region_desc,region,sex,prep_fsw,prep_migrant,prep_msm,prep_msw,prep_prisoner,prep_pwid,prep_tg,prep_tgsw,prep_all_kps,prep_others,prep_total
0,2023,11,กระบี่,ภาคใต้ตอนบน (ฝั่งอ่าวไทย/อันดามัน),ภาคใต้,Krabi,Upper Southern,South,female,0,0,0,0,0,0,0,0,0,4,4
1,2023,11,กระบี่,ภาคใต้ตอนบน (ฝั่งอ่าวไทย/อันดามัน),ภาคใต้,Krabi,Upper Southern,South,male,0,0,22,0,0,0,0,0,22,3,25
2,2023,13,กรุงเทพมหานคร,กรุงเทพมหานคร (กทม.),ภาคกลาง,Bangkok,Bangkok Metropolitan (BMA),Central,female,452,0,20,0,0,7,7,0,486,608,1094
3,2023,13,กรุงเทพมหานคร,กรุงเทพมหานคร (กทม.),ภาคกลาง,Bangkok,Bangkok Metropolitan (BMA),Central,male,0,0,14094,1088,1,91,1111,208,16593,379,16972
4,2023,5,กาญจนบุรี,ภาคตะวันตก,ภาคกลาง,Kanchanaburi,Western Region,Central,female,2,0,0,0,0,0,0,0,2,5,7


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   year                   434 non-null    string
 1   health_region          434 non-null    string
 2   province_th            434 non-null    string
 3   health_region_desc_th  434 non-null    string
 4   region_th              434 non-null    string
 5   province               434 non-null    string
 6   health_region_desc     434 non-null    string
 7   region                 434 non-null    string
 8   sex                    434 non-null    string
 9   prep_fsw               434 non-null    int64 
 10  prep_migrant           434 non-null    int64 
 11  prep_msm               434 non-null    int64 
 12  prep_msw               434 non-null    int64 
 13  prep_prisoner          434 non-null    int64 
 14  prep_pwid              434 non-null    int64 
 15  prep_tg                

None

In [14]:
# @title Merge dataset (Wide Format)

col_to_merge = ['year', 'health_region', 'province_th', 'health_region_desc_th', 'region_th', 'province', 'health_region_desc', 'region', 'sex']
df_hiv_pop = df_test_pop.merge(df_reach_pop, on=col_to_merge, how='left').merge(df_prep_pop, on=col_to_merge, how='left')

#cols_to_fix = df_hiv_pop.columns[33:]
#df_hiv_pop[cols_to_fix] = df_hiv_pop[cols_to_fix].fillna(0).astype('int64')

display(df_hiv_pop.info())
df_hiv_pop.to_excel('hiv_pop.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     770 non-null    string 
 1   health_region            770 non-null    string 
 2   province_th              770 non-null    string 
 3   health_region_desc_th    770 non-null    string 
 4   region_th                770 non-null    string 
 5   province                 770 non-null    string 
 6   health_region_desc       770 non-null    string 
 7   region                   770 non-null    string 
 8   sex                      770 non-null    string 
 9   test_fsw                 770 non-null    int64  
 10  test_migrant             770 non-null    int64  
 11  test_msm                 770 non-null    int64  
 12  test_msw                 770 non-null    int64  
 13  test_partner_of_hiv      770 non-null    int64  
 14  test_prisoner            7

None

In [15]:
# @title Merge dataset (Long Format)

col_to_merge = ['year', 'health_region', 'province_th', 'health_region_desc_th', 'region_th', 'province', 'health_region_desc', 'region', 'sex']
df_hiv_pop = df_test_pop.merge(df_reach_pop, on=col_to_merge, how='left').merge(df_prep_pop, on=col_to_merge, how='left')

#cols_to_fix = df_hiv_pop.columns[33:]
#df_hiv_pop[cols_to_fix] = df_hiv_pop[cols_to_fix].fillna(0).astype('int64')

df_hiv_pop = df_hiv_pop.drop(columns=['test_total','positive_total','reach_total','prep_total'])

# Melt to Long Format (Unpivot Columns)
df_hiv_pop = df_hiv_pop.melt(
    id_vars=['year', 'health_region', 'province_th', 'health_region_desc_th', 'region_th', 'province', 'health_region_desc', 'region', 'sex'],
    var_name='temp_column',
    value_name='count'
)

split_data = df_hiv_pop['temp_column'].str.split('_', expand=True)
df_hiv_pop['category'] = split_data[0]

df_hiv_pop['population'] = split_data.iloc[:, 1:].fillna('').agg('_'.join, axis=1).str.strip('_')

# Pivot to Wide Format
df_hiv_pop = df_hiv_pop.pivot(
    index=['year', 'health_region', 'province_th', 'health_region_desc_th', 'region_th', 'province', 'health_region_desc', 'region', 'sex', 'population'],
    columns='category',
    values='count'
).reset_index()
df_hiv_pop.columns.name = None

df_hiv_pop['population'] = df_hiv_pop['population'].astype('string')
# Sort Columns
desired_order = ['year', 'health_region', 'province_th', 'health_region_desc_th', 'region_th', 'province', 'health_region_desc', 'region', 'sex', 'population',
                 'test','positive','reach','prep']
actual_columns = [col for col in desired_order if col in df_hiv_pop.columns]
df_hiv_pop = df_hiv_pop[actual_columns]

display(df_hiv_pop.info())
df_hiv_pop.to_excel('hiv_pop_long.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8470 entries, 0 to 8469
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   8470 non-null   string 
 1   health_region          8470 non-null   string 
 2   province_th            8470 non-null   string 
 3   health_region_desc_th  8470 non-null   string 
 4   region_th              8470 non-null   string 
 5   province               8470 non-null   string 
 6   health_region_desc     8470 non-null   string 
 7   region                 8470 non-null   string 
 8   sex                    8470 non-null   string 
 9   population             8470 non-null   string 
 10  test                   8470 non-null   float64
 11  positive               8470 non-null   float64
 12  reach                  3520 non-null   float64
 13  prep                   4340 non-null   float64
dtypes: float64(4), string(10)
memory usage: 926.5 KB


None

In [9]:
# @title HIV Testing by Age
path = "/content/dads5001_midterm_project/rd_testing_agegr/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_test_age_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[2]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.contains('testing')) & (df[1].str.contains('Female')),
      (df[0].str.contains('testing')) & (df[1].str.contains('Male')),
      (df[0].str.contains('testing')) & (df[1].str.contains('Total')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Female')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Male')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Total'))
  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'test_female', 'test_male', 'test_total',
            'positive_female', 'positive_male', 'positive_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_test_age_combine is None):
    df_test_age_combine = df
  else:
    df_test_age_combine = pd.concat([df_test_age_combine, df], ignore_index=True)

display(df_test_age_combine.groupby(['year','quarter'])['positive_grand_total'].count())

# Transform accumulated data to total year summary
df_test_age = df_test_age_combine[df_test_age_combine['q'] == 'Q4'].copy()
df_test_age_temp1 = df_test_age_combine[(df_test_age_combine['q'] == 'Q1') & (df_test_age_combine['year'].isin(df_test_age['year'].unique()))].copy()

next_years_int = df_test_age['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_test_age_temp2 = df_test_age_combine[(df_test_age_combine['q'] == 'Q1') & (df_test_age_combine['year'].isin(next_years_str))].copy()

df_test_age = df_test_age.set_index(['year', 'province'])
df_test_age_temp1 = df_test_age_temp1.set_index(['year', 'province'])

df_test_age_temp2['year'] = df_test_age_temp2['year'].astype(int) - 1
df_test_age_temp2['year'] = df_test_age_temp2['year'].astype(str)
df_test_age_temp2 = df_test_age_temp2.set_index(['year', 'province'])

cols_to_cal = df_test_age.columns[3:]
#df_test_age[cols_to_cal] = (df_test_age[cols_to_cal] - df_test_age_temp1[cols_to_cal]) + df_test_age_temp2[cols_to_cal]
df_test_age[cols_to_cal] = df_test_age[cols_to_cal].sub(df_test_age_temp1[cols_to_cal], fill_value=0).add(df_test_age_temp2[cols_to_cal], fill_value=0)

df_test_age = df_test_age.reset_index(drop=False)
df_test_age = df_test_age.drop(columns=['q','quarter'])
df_test_age.iloc[:, 3:] = df_test_age.iloc[:, 3:].clip(lower=0)

# Delete temp variables
del df_test_age_temp1
del df_test_age_temp2
gc.collect()

# ต้องเช็กก่อนว่า header ต้องใช้คำว่าอะไร และเปลี่ยนเงื่อนไข
test_male = [col for col in df_test_age.columns if col.startswith('test_male_') and 'total' not in col]
test_female = [col for col in df_test_age.columns if col.startswith('test_female_') and 'total' not in col]
positive_male = [col for col in df_test_age.columns if col.startswith('positive_male_') and 'total' not in col]
positive_female = [col for col in df_test_age.columns if col.startswith('positive_female_') and 'total' not in col]

df_test_age['test_male_total'] = df_test_age[test_male].sum(axis=1)
df_test_age['test_female_total'] = df_test_age[test_female].sum(axis=1)
df_test_age['positive_male_total'] = df_test_age[positive_male].sum(axis=1)
df_test_age['positive_female_total'] = df_test_age[positive_female].sum(axis=1)

df_test_age = df_test_age.drop(columns=['test_grand_total','positive_grand_total'])

# Melt to Long Format (Unpivot Columns)
df_test_age = df_test_age.melt(
    id_vars=['year', 'province', 'health_region'],
    var_name='temp_column',
    value_name='count'
)

split_data = df_test_age['temp_column'].str.split('_', expand=True)
df_test_age['sex'] = split_data[1]

df_test_age['category'] = split_data[0] + '_' + split_data.iloc[:, 2:].fillna('').agg('_'.join, axis=1).str.strip('_')
df_test_age = df_test_age[['year', 'province', 'health_region', 'sex', 'category', 'count']]

# Pivot to Wide Format
df_test_age = df_test_age.pivot(
    index=['year', 'province', 'health_region', 'sex'],
    columns='category',
    values='count'
).reset_index()
df_test_age.columns.name = None

# Change Type
for col in df_test_age.iloc[0:,0:4].columns: df_test_age[col] = df_test_age[col].astype('string')
#for col in df_test_age.iloc[0:,4:].columns: df_test_age[col] = df_test_age[col].astype('int64')

# Sort Columns
desired_order = ['year', 'province', 'health_region', 'sex',
                 'test_<1', 'test_1-4', 'test_5-9', 'test_10-14', 'test_15-19', 'test_20-24', 'test_25-49', 'test_50+', 'test_total',
                 'positive_<1', 'positive_1-4', 'positive_5-9', 'positive_10-14', 'positive_15-19', 'positive_20-24', 'positive_25-49', 'positive_50+', 'positive_total'
]
actual_columns = [col for col in desired_order if col in df_test_age.columns]
df_test_age = df_test_age[actual_columns]

display(df_test_age.head())
display(df_test_age.info())

df_test_age.to_excel('test_age.xlsx', index=False)

Total files: 7
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2023Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2024Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2025Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2025Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2023Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2026Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2024Q4_20260215.xlsx


year  quarter
2023  2023Q1     77
      2023Q4     77
2024  2024Q1     77
      2024Q4     77
2025  2025Q1     77
      2025Q4     77
2026  2026Q1     77
Name: positive_grand_total, dtype: int64

,year,province,health_region,sex,test_<1,test_1-4,test_5-9,test_10-14,test_15-19,test_20-24,...,test_total,positive_<1,positive_1-4,positive_5-9,positive_10-14,positive_15-19,positive_20-24,positive_25-49,positive_50+,positive_total
0,2023,กระบี่,11,female,9,74,96,156,471,786,...,8146,0,0,1,0,3,5,31,16,56
1,2023,กระบี่,11,male,22,106,149,176,331,466,...,6770,0,0,1,0,3,17,64,12,97
2,2023,กรุงเทพมหานคร,13,female,19,148,125,634,3670,8269,...,73424,0,4,0,1,41,98,532,206,882
3,2023,กรุงเทพมหานคร,13,male,18,156,227,564,10368,15913,...,107995,0,0,2,5,168,660,2166,370,3371
4,2023,กาญจนบุรี,5,female,2,24,47,119,1105,743,...,9230,0,0,0,1,10,13,49,30,103


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            462 non-null    string
 1   province        462 non-null    string
 2   health_region   462 non-null    string
 3   sex             462 non-null    string
 4   test_<1         462 non-null    int64 
 5   test_1-4        462 non-null    int64 
 6   test_5-9        462 non-null    int64 
 7   test_10-14      462 non-null    int64 
 8   test_15-19      462 non-null    int64 
 9   test_20-24      462 non-null    int64 
 10  test_25-49      462 non-null    int64 
 11  test_50+        462 non-null    int64 
 12  test_total      462 non-null    int64 
 13  positive_<1     462 non-null    int64 
 14  positive_1-4    462 non-null    int64 
 15  positive_5-9    462 non-null    int64 
 16  positive_10-14  462 non-null    int64 
 17  positive_15-19  462 non-null    int64 
 18  positive_2

None

In [10]:
df_test_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            462 non-null    string
 1   province        462 non-null    string
 2   health_region   462 non-null    string
 3   sex             462 non-null    string
 4   test_<1         462 non-null    int64 
 5   test_1-4        462 non-null    int64 
 6   test_5-9        462 non-null    int64 
 7   test_10-14      462 non-null    int64 
 8   test_15-19      462 non-null    int64 
 9   test_20-24      462 non-null    int64 
 10  test_25-49      462 non-null    int64 
 11  test_50+        462 non-null    int64 
 12  test_total      462 non-null    int64 
 13  positive_<1     462 non-null    int64 
 14  positive_1-4    462 non-null    int64 
 15  positive_5-9    462 non-null    int64 
 16  positive_10-14  462 non-null    int64 
 17  positive_15-19  462 non-null    int64 
 18  positive_2

In [11]:
df_prep_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   year                   434 non-null    string
 1   health_region          434 non-null    string
 2   province_th            434 non-null    string
 3   health_region_desc_th  434 non-null    string
 4   region_th              434 non-null    string
 5   province               434 non-null    string
 6   health_region_desc     434 non-null    string
 7   region                 434 non-null    string
 8   sex                    434 non-null    string
 9   prep_fsw               434 non-null    int64 
 10  prep_migrant           434 non-null    int64 
 11  prep_msm               434 non-null    int64 
 12  prep_msw               434 non-null    int64 
 13  prep_prisoner          434 non-null    int64 
 14  prep_pwid              434 non-null    int64 
 15  prep_tg                

In [12]:
# @title PrEP Cur by Age
path = "/content/dads5001_midterm_project/rd_prep_agegr/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_prep_age_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[3]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Female')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Male')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Total')),
  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'prep_female', 'prep_male', 'prep_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_prep_age_combine is None):
    df_prep_age_combine = df
  else:
    df_prep_age_combine = pd.concat([df_prep_age_combine, df], ignore_index=True)

display(df_prep_age_combine.groupby(['year','quarter'])['prep_grand_total'].count())

# Transform accumulated data to total year summary
df_prep_age = df_prep_age_combine[df_prep_age_combine['q'] == 'Q4'].copy()
df_prep_age_temp1 = df_prep_age_combine[(df_prep_age_combine['q'] == 'Q1') & (df_prep_age_combine['year'].isin(df_prep_age['year'].unique()))].copy()

next_years_int = df_prep_age['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_prep_age_temp2 = df_prep_age_combine[(df_prep_age_combine['q'] == 'Q1') & (df_prep_age_combine['year'].isin(next_years_str))].copy()

df_prep_age = df_prep_age.set_index(['year', 'province'])
df_prep_age_temp1 = df_prep_age_temp1.set_index(['year', 'province'])

df_prep_age_temp2['year'] = df_prep_age_temp2['year'].astype(int) - 1
df_prep_age_temp2['year'] = df_prep_age_temp2['year'].astype(str)
df_prep_age_temp2 = df_prep_age_temp2.set_index(['year', 'province'])

cols_to_cal = df_prep_age.columns[3:]
#df_prep_age[cols_to_cal] = (df_prep_age[cols_to_cal] - df_prep_age_temp1[cols_to_cal]) + df_prep_age_temp2[cols_to_cal]
df_prep_age[cols_to_cal] = df_prep_age[cols_to_cal].sub(df_prep_age_temp1[cols_to_cal], fill_value=0).add(df_prep_age_temp2[cols_to_cal], fill_value=0)

df_prep_age = df_prep_age.reset_index(drop=False)
df_prep_age = df_prep_age.drop(columns=['q','quarter'])
df_prep_age.iloc[:, 3:] = df_prep_age.iloc[:, 3:].clip(lower=0)

# Delete temp variables
del df_prep_age_temp1
del df_prep_age_temp2
gc.collect()

prep_male = [col for col in df_prep_age.columns if col.startswith('prep_male_') and 'total' not in col]
prep_female = [col for col in df_prep_age.columns if col.startswith('prep_female_') and 'total' not in col]

df_prep_age['prep_male_total'] = df_prep_age[prep_male].sum(axis=1)
df_prep_age['prep_female_total'] = df_prep_age[prep_female].sum(axis=1)

df_prep_age = df_prep_age.drop(columns='prep_grand_total')

# Melt to Long Format (Unpivot Columns)
df_prep_age = df_prep_age.melt(
    id_vars=['year', 'province', 'health_region'],
    var_name='temp_column',
    value_name='count'
)

split_data = df_prep_age['temp_column'].str.split('_', expand=True)
df_prep_age['sex'] = split_data[1]

df_prep_age['category'] = split_data.iloc[:, 2:].fillna('').agg('_'.join, axis=1).str.strip('_')
df_prep_age = df_prep_age[['year', 'province', 'health_region', 'sex', 'category', 'count']]

# Pivot to Wide Format
df_prep_age = df_prep_age.pivot(
    index=['year', 'province', 'health_region', 'sex'],
    columns='category',
    values='count'
).reset_index()
df_prep_age.columns.name = None

# Change Type
for col in df_prep_age.iloc[0:,0:4].columns: df_prep_age[col] = df_prep_age[col].astype('string')
for col in df_prep_age.iloc[0:,4:].columns: df_prep_age[col] = df_prep_age[col].astype('int64')

# Sort Columns
desired_order = ['year', 'province', 'health_region', 'sex', '<1', '1-4', '5-9', '10-14', '15-19', '20-24', '25-49', '50+', 'total']
actual_columns = [col for col in desired_order if col in df_prep_age.columns]
df_prep_age = df_prep_age[actual_columns]

display(df_prep_age.head())
display(df_prep_age.info())
df_prep_age.to_excel('prep_age.xlsx', index=False)

Total files: 7
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2024Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2024Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2025Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2023Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2023Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2025Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2026Q1_20260215.xlsx


year  quarter
2023  2023Q1     54
      2023Q4     67
2024  2024Q1     74
      2024Q4     74
2025  2025Q1     72
      2025Q4     76
2026  2026Q1     69
Name: prep_grand_total, dtype: int64

,year,province,health_region,sex,<1,1-4,5-9,10-14,15-19,20-24,25-49,50+,total
0,2023,กระบี่,11,female,0,0,0,0,0,0,4,0,4
1,2023,กระบี่,11,male,0,0,0,0,0,0,25,0,25
2,2023,กรุงเทพมหานคร,13,female,0,0,0,3,69,252,737,33,1094
3,2023,กรุงเทพมหานคร,13,male,21,0,0,3,461,2951,13086,450,16972
4,2023,กาญจนบุรี,5,female,0,0,0,0,0,0,7,0,7


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           434 non-null    string
 1   province       434 non-null    string
 2   health_region  434 non-null    string
 3   sex            434 non-null    string
 4   <1             434 non-null    int64 
 5   1-4            434 non-null    int64 
 6   5-9            434 non-null    int64 
 7   10-14          434 non-null    int64 
 8   15-19          434 non-null    int64 
 9   20-24          434 non-null    int64 
 10  25-49          434 non-null    int64 
 11  50+            434 non-null    int64 
 12  total          434 non-null    int64 
dtypes: int64(9), string(4)
memory usage: 44.2 KB


None

In [13]:
df_prep_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           434 non-null    string
 1   province       434 non-null    string
 2   health_region  434 non-null    string
 3   sex            434 non-null    string
 4   <1             434 non-null    int64 
 5   1-4            434 non-null    int64 
 6   5-9            434 non-null    int64 
 7   10-14          434 non-null    int64 
 8   15-19          434 non-null    int64 
 9   20-24          434 non-null    int64 
 10  25-49          434 non-null    int64 
 11  50+            434 non-null    int64 
 12  total          434 non-null    int64 
dtypes: int64(9), string(4)
memory usage: 44.2 KB
